In [7]:
# code by Tae Hwan Jung @graykode
# Reference : https://github.com/hunkim/PyTorchZeroToAll/blob/master/14_2_seq2seq_att.py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import psycopg2 as pg
import pandas as pd

# S: Symbol that shows starting of decoding input
# E: Symbol that shows starting of decoding output
# P: Symbol that will fill in blank sequence if current batch data size is short than time steps
class DatabaseConnection:
    def __init__(self):
        self.conn = self.connect()
    def connect(self):
        try:
            self.conn = pg.connect(
                "dbname='stonksdb' user='postgres' host='localhost' password='admin'")
            print("Connection made succ")
            return self.conn
        except:
            print("I am unable to connect to the database")
            return None
    def close(self):
        self.conn.close()
    
    def GetConnector(self):
        return self.conn
    ### send query to database
    def query(self, query):
        cur = self.conn.cursor()
        cur.execute(query)
        return cur.fetchall()

class DatasetAccess:
    def __init__(self):
        self.conn = DatabaseConnection()
    def getNcompanies(self, N):
        AllCompanies = self.conn.query("SELECT * FROM dataset limit "+ str(N)+"")
        return AllCompanies
    def getStockFromSymbol(self, StockSymbol, column = '*'):
        company = self.conn.query("SELECT * FROM dataset WHERE symbol = '" + StockSymbol + "'")
        self.getStockFromCompany(company, column)
        return company
    def getStockFromCompany(self, companies, column = '*'):
        result = []
        for company in companies:
            result.append(self.conn.query("SELECT "+self.convertListToString(column)+" FROM stock WHERE identifier = '" + str(company[0]) + "'"))
        return result
    def convertListToString(self, column):
        if type(column) != list:
            return column
        result = ''
        for item in column:
            result += item + ', '
        return result[:-2]
    def getStockDFFromCompany(self, companies, column = '*'):
        result = []
        for company in companies:
            result.append(pd.read_sql("SELECT "+self.convertListToString(column)+" FROM stock WHERE identifier = '" + str(company[0]) + "'", self.conn.GetConnector()))
        return result
    def GetAllStocksAsDF(self):
        PandaStock = pd.read_sql('SELECT * FROM stock', self.conn.GetConnector())
        print(PandaStock)
    
def GetSingleStockDF():
    dbAccess = DatasetAccess()
    comp = dbAccess.getNcompanies(2)
    return dbAccess.getStockDFFromCompany(comp, column="close")

In [10]:
def make_batch():
    replica_input = train
    replica_input = [np.array([replica_input])]
    replica_Input = [n for n in replica_input[0]]
    print("____________-")
    replica_output = target
    replica_output = [np.array([replica_output])]
    replica_Output = [n for n in replica_output[0]]
    print("____________-")
    replica_target = target
    replica_target = [np.array([replica_target])]
    replica_Target = [n for n in replica_target[0]]
    
    #input_batch = [np.eye(n_class)[[word_dict[n] for n in sentences[0].split()]]] # takes the number and one hot encodes it
    #print(input_batch)
    #print(replica_Output)
    #output_batch = [np.eye(n_class)[[word_dict[n] for n in sentences[1].split()]]] # takes the number and one hot encodes it
    #target_batch = [[word_dict[n] for n in sentences[2].split()]] # takes the number
    # print(input_batch)
    # print([np.array([replica_input])])
    # make tensor
    # return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)
    return torch.FloatTensor(replica_Input), torch.FloatTensor(replica_Output), torch.FloatTensor(replica_Target)

make_batch()

____________-
____________-


C:\Users\Jeppe\AppData\Local\Temp\ipykernel_612\2894743647.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.FloatTensor(replica_Input), torch.FloatTensor(replica_Output), torch.FloatTensor(replica_Target)


(tensor([[[0.9900, 0.9904, 0.9912, 0.9904, 0.9908, 0.9896, 0.9908, 0.9904,
           0.9896, 0.9896, 0.9900, 0.9888, 0.9896, 0.9892, 0.9892, 0.9884,
           0.9888, 0.9884, 0.9828, 0.9812, 0.9804, 0.9792, 0.9796, 0.9800,
           0.9792, 0.9788, 0.9788, 0.9792, 0.9792, 0.9796, 0.9788, 0.9800,
           0.9792, 0.9792, 0.9796, 0.9784, 0.9788, 0.9788, 0.9800, 0.9792,
           0.9788, 0.9788, 0.9792, 0.9780, 0.9776, 0.9776, 0.9760, 0.9768,
           0.9772, 0.9764],
          [0.9832, 0.9836, 0.9840, 0.9840, 0.9824, 0.9832, 0.9832, 0.9840,
           0.9844, 0.9848, 0.9848, 0.9844, 0.9860, 0.9868, 0.9872, 0.9864,
           0.9852, 0.9848, 0.9844, 0.9856, 0.9856, 0.9860, 0.9864, 0.9848,
           0.9852, 0.9860, 0.9864, 0.9868, 0.9868, 0.9880, 0.9888, 0.9892,
           0.9884, 0.9880, 0.9884, 0.9880, 0.9884, 0.9888, 0.9884, 0.9896,
           0.9892, 0.9896, 0.9900, 0.9896, 0.9916, 0.9916, 0.9908, 0.9900,
           0.9904, 0.9912],
          [0.9739, 0.9731, 0.9739, 0.9739, 0

In [ ]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)

        # Linear for attention
        self.attn = nn.Linear(n_hidden, n_hidden)
        self.out = nn.Linear(n_hidden * 2, n_class)

    def forward(self, enc_inputs, hidden, dec_inputs):
        enc_inputs = enc_inputs.transpose(0, 1)  # enc_inputs: [n_step(=n_step, time step), batch_size, n_class]
        dec_inputs = dec_inputs.transpose(0, 1)  # dec_inputs: [n_step(=n_step, time step), batch_size, n_class]

        # enc_outputs : [n_step, batch_size, num_directions(=1) * n_hidden], matrix F
        # enc_hidden : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        enc_outputs, enc_hidden = self.enc_cell(enc_inputs, hidden)

        trained_attn = []
        hidden = enc_hidden
        n_step = len(dec_inputs)
        model = torch.empty([n_step, 1, n_class])

        for i in range(n_step):  # each time step
            # dec_output : [n_step(=1), batch_size(=1), num_directions(=1) * n_hidden]
            # hidden : [num_layers(=1) * num_directions(=1), batch_size(=1), n_hidden]
            dec_output, hidden = self.dec_cell(dec_inputs[i].unsqueeze(0), hidden)
            attn_weights = self.get_att_weight(dec_output, enc_outputs)  # attn_weights : [1, 1, n_step]
            trained_attn.append(attn_weights.squeeze().data.numpy())

            # matrix-matrix product of matrices [1,1,n_step] x [1,n_step,n_hidden] = [1,1,n_hidden]
            context = attn_weights.bmm(enc_outputs.transpose(0, 1))
            dec_output = dec_output.squeeze(0)  # dec_output : [batch_size(=1), num_directions(=1) * n_hidden]
            context = context.squeeze(1)  # [1, num_directions(=1) * n_hidden]
            model[i] = self.out(torch.cat((dec_output, context), 1))

        # make model shape [n_step, n_class]
        return model.transpose(0, 1).squeeze(0), trained_attn

    def get_att_weight(self, dec_output, enc_outputs):  # get attention weight one 'dec_output' with 'enc_outputs'
        n_step = len(enc_outputs)
        attn_scores = torch.zeros(n_step)  # attn_scores : [n_step]

        for i in range(n_step):
            attn_scores[i] = self.get_att_score(dec_output, enc_outputs[i])

        # Normalize scores to weights in range 0 to 1
        return F.softmax(attn_scores).view(1, 1, -1)

    def get_att_score(self, dec_output, enc_output):  # enc_outputs [batch_size, num_directions(=1) * n_hidden]
        score = self.attn(enc_output)  # score : [batch_size, n_hidden]
        return torch.dot(dec_output.view(-1), score.view(-1))  # inner product make scalar value

In [13]:
if __name__ == '__main__':
    n_step = 5 # number of cells(= number of Step)
    n_hidden = 128 # number of hidden units in one cell
    closingData = GetSingleStockDF()[1][:1000]
    n_class = 50
    chunks = 10

    closingData = np.array(closingData).flatten()
    closingData = closingData / closingData.max()
    data = np.array_split(closingData, chunks)

    train = [d[:50] for d in data]
    target = [d[50:] for d in data]
    
    hidden = torch.zeros(1, 1, n_hidden)

    model = Attention()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    input_batch, output_batch, target_batch = make_batch()
    
    # Train
    for epoch in range(2000):
        optimizer.zero_grad()
        output, _ = model(input_batch, hidden, output_batch)

        loss = criterion(output, target_batch.squeeze(0))
        if (epoch + 1) % 400 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

        loss.backward()
        optimizer.step()


Connection made succ


C:\Users\Jeppe\AppData\Roaming\Python\Python310\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\Jeppe\AppData\Roaming\Python\Python310\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\Jeppe\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
C:\Users\Jeppe\AppData\Local\Temp\ipykernel_612\938175066.py:48: UserWar

____________-
____________-
Epoch: 0400 cost = 21645.035156
Epoch: 0800 cost = 20399.583984
Epoch: 1200 cost = 28746.265625
Epoch: 1600 cost = 27468.175781
Epoch: 2000 cost = 28559.324219
